In [ ]:
import xarray as xr
import pandas as pd
import paraguayfloodspy.visualize as viz
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

The seasonal forecasts were downloaded from the IRI data library to speed computation

In [ ]:
url = '(https://iridl.ldeo.columbia.edu/expert/90.0//percentile/parameter/SOURCES/.IRI/.FD/.Flexible_Forecast/.precipitation/exch/100.0/div/2/mul/1/sub/erfinv/clim_stddev/mul/2/sqrt/mul/clim_mean/add/X/-66.25/-48.75/RANGE/Y/-39.25/-16.75/RANGE/dup/fcst_mean/sub/fcst_stddev/div/2/sqrt/div/erf/-1/mul/1/add/2/div/(exceeding)//proba/parameter/(exceeding)/ne/%7B-1/mul/1/add%7Dif/S//fullname/(Month%20Forecast%20Issued)/def/pop/(long_name)/(Probability%20of%20)//proba/get_parameter/append/(%20)/append//percentile/get_parameter/s==/append/(th%20%25-ile)/append/def/L/2.5/VALUE/S/670.0/VALUE/gridtable.tsv'
print(url)

In [ ]:
savefigs=True

In [ ]:
df = pd.read_csv("../data/given/SeasonalForecast.tsv", skiprows=2, 
                 delim_whitespace=True, names=['lon', 'lat', 'prob_exceed'],
                index_col=['lat', 'lon'])
df = df['prob_exceed'].to_xarray()
X,Y = np.meshgrid(df.lon, df.lat)
X = X % 360

Where was there an observed exceedance?

In [ ]:
prcp = xr.open_dataset("../data/derived/precip.nc")['raw']
prcp = prcp.sel(lon = slice(X.min(), X.max()), lat = slice(Y.min(), Y.max()))
prcp = prcp.sel(time = prcp['time.season'] == 'DJF')
year_adj = prcp['time.year']
year_adj[np.where(prcp['time.month'].values == 12)] += 1
prcp['year_adj'] = year_adj
prcp_year = prcp.groupby('year_adj').mean(dim='time')
prcp_q90 = prcp_year.quantile(0.90, dim='year_adj')
prcp_exceed = prcp_year.sel(year_adj=2016) > prcp_q90

In [ ]:
Xh,Yh = np.meshgrid(prcp_exceed.lon, prcp_exceed.lat)
hatch = np.ma.masked_invalid(prcp_exceed.values)
hatch = np.ma.masked_less(hatch, True)

In [ ]:
fig, ax = viz.SetupAxes(figsize=(5.5,8), ncol=1, nax=1,proj=ccrs.PlateCarree())
fig.subplots_adjust(right=0.925)
cax1 = fig.add_axes([0.95, 0.15, 0.035, 0.7])
sub = np.ma.masked_invalid((df/(1-df)) / (.1/(1-.1)))
C = ax.pcolormesh(X,Y, sub, transform=ccrs.PlateCarree(), 
                  cmap="gist_rainbow", vmin=0, vmax=7)
ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')
cb = plt.colorbar(C, cax=cax1)
cb.set_label("Odds Ratio", rotation=270)
cb.ax.get_yaxis().labelpad = 15
viz.FormatAxes(ax, ticks=[np.linspace(-180, 180, 73), np.linspace(-90, 90, 37)], 
               extent = [np.min(X), np.max(X), np.min(Y), np.max(Y)])
if savefigs:
    fig.savefig("../figs/SeasonalForecast.pdf", bbox_inches='tight')